# Actions: 

Missing_Value_Imputation :fill- 0

Future Engineering: Chi-square & Feature_importance(GradientBoostingClassifier)

Sampling: Stratified Random Sampling

Train & Test Split: 33

Machine Learning Model: GradientBoostingClassifier

Model Assessments: ROC_AUC_SCORE

Hyper_parameter Tuning: n_estimator, loss, Max_depth

# Data Extraction

In [ ]:
#importing required library's
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing

In [ ]:
#importing train & test data
df_train=pd.read_csv('/kaggle/input/av-jobhackthon-credit-card-lead-prediction/train_hack.csv')
df_test=pd.read_csv('/kaggle/input/av-jobhackthon-credit-card-lead-prediction/test_hack.csv')
sample_submission=pd.read_csv('/kaggle/input/av-jobhackthon-credit-card-lead-prediction/sample_submission_eyYijxG_hack.csv')

In [ ]:
df_train=df_train[~df_train.ID.isin(['a'])]
df_test=df_test[~df_test.ID.isin(['a'])]
sample_submission=sample_submission[~sample_submission.ID.isin(['a'])]

# EDA(Exploratory Data Analysis)

In [ ]:
#Basic checks
print("******** No.of Row's and Column's in Dataframe********* \n ",df_train.shape,
     "\n \n **************Columns in Dataframe*********** \n ",df_train.columns,
     "\n \n **************Top 5 Row's in Dataframe *********** \n ",df_train.dtypes)

In [ ]:
# Top 5 rows
df_train.head()

In [ ]:
# bottom 5 row
df_train.tail()

# Duplicates Check

In [ ]:
duplicate = df_train[df_train.duplicated()]
duplicate

# Missing Values Check


In [ ]:
# Identifying missing values
miss_check=pd.DataFrame(df_train.isnull().sum())

# Calculating missing value percentage 
miss_check.reset_index(inplace=True)
miss_check.columns=['features','missing_count']
miss_check['missing_value_percentage']=(miss_check.missing_count/df_train.shape[0])*100
miss_check.plot.barh(x='features',y='missing_count',figsize=(8,5))

# Missing value features
miss_check[miss_check.missing_value_percentage>0]

# Remove Unwanted columns

In [ ]:
df_train.drop(['ID'],axis=1,inplace=True)

# Missing Value Impution

In [ ]:
# Replacing missing values with Zero
df_train.fillna(0,inplace=True)

# Data Converstion/Label encoding

In [ ]:
#Creating Dummy variables for Categorical variables
df_train2=pd.get_dummies(df_train, columns = ['Gender', 'Region_Code', 'Occupation', 'Channel_Code',
       'Credit_Product', 'Is_Active',],drop_first=True)
X = df_train2.loc[:, df_train2.columns != 'Is_Lead']
y = df_train2.loc[:, df_train2.columns == 'Is_Lead']

# Feature Engineering

In [ ]:
#Seleting chi-square test for feature engineering
bestfeature=SelectKBest(score_func=chi2,k=10)
fit=bestfeature.fit(X,y)
df_p_score=pd.DataFrame(fit.pvalues_)
df_columns=pd.DataFrame(X.columns)
feature_score=pd.concat([df_columns,df_p_score],axis=1)
feature_score.columns=['features','P_score']

In [ ]:
# Printing the score
feature_score.sort_values(by=['P_score'],ascending=True)

In [ ]:
feature_score.plot.barh(x='features',y='P_score',figsize=(10,15))

In [ ]:
# Taking cutoff level for features is 0.08539033
new_features=feature_score[feature_score.P_score<0.08539033]

# Taking features for model build
X1=X[new_features.features]

# before and after features engineering task no.of features
print("******** After Feature engineering no.of features for model******** \n ",len(new_features.features),"\n \n******** Before feature engineering no.of features for model******** \n \n  ",len(X.columns))

# Sampling & Spliting

In [ ]:
# Stratified Random sampling & Test data 33%
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.33, random_state=42,stratify=y)

# Model Training

In [ ]:
# Selected GradientBoostingClassifier for model build
print(len(X1.columns))
gb_classifier=GradientBoostingClassifier(loss='exponential',n_estimators=100,max_depth=5,max_features='auto')
gb_classifier.fit(X_train, y_train)
ytrain_pred = gb_classifier.predict_proba(X_train)
print('GradientBoostingClassifier train roc-auc: {}'.format(roc_auc_score(y_train, ytrain_pred[:,1])))
ytest_pred = gb_classifier.predict_proba(X_test)
print('GradientBoostingClassifier test roc-auc: {}'.format(roc_auc_score(y_test, ytest_pred[:,1])))


In [ ]:
# Feature engineering from model output.
features_score=pd.DataFrame(gb_classifier.feature_importances_,columns=['features_score'])
features=pd.DataFrame(X_train.columns,columns=['features'])
gb_features=pd.concat([features,features_score],axis=1)
print(gb_features.sort_values(by='features_score',ascending=False))

# The cutoff features_score>=0.000257 giving better results
gb_f_features=gb_features[gb_features.features_score>=0.000275]

# Taking features for model build
X1=X[gb_f_features.features]

# before and after features engineering task no.of features
print("******** After Feature engineering no.of features for model******** \n ",len(gb_f_features.features),"\n \n******** Before feature engineering no.of features for model******** \n \n  ",len(new_features.features))


In [ ]:
gb_f_features.plot.barh(x='features',y='features_score',figsize=(10,15))

# Re-Train the Model

In [ ]:
# Stratified Random sampling & Test data 33%
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.33, random_state=42,stratify=y)

In [ ]:
gb_classifier=GradientBoostingClassifier(loss='exponential',n_estimators=100,max_depth=5,max_features='auto')
gb_classifier.fit(X_train, y_train)
ytrain_pred = gb_classifier.predict_proba(X_train)
print('GradientBoostingClassifier train roc-auc: {}'.format(roc_auc_score(y_train, ytrain_pred[:,1])))
ytest_pred = gb_classifier.predict_proba(X_test)
print('GradientBoostingClassifier test roc-auc: {}'.format(roc_auc_score(y_test, ytest_pred[:,1])))

# Prediction for Test data

In [ ]:
#Features selection for predictions
filter_final_cols=X1.columns
print("*********** No.of feature selected for preditions ************\n",len(filter_final_cols))

In [ ]:
#Missing value imputatiion
df_test.fillna(0,inplace=True)

#Data conversions/Label encoding
df_dummy_test = pd.get_dummies(df_test, columns = ['Gender', 'Region_Code', 'Occupation', 'Channel_Code',
       'Credit_Product', 'Is_Active'],drop_first=True)

# Features selection
X_test_prd=df_dummy_test[filter_final_cols]

# probability prediction
pred=gb_classifier.predict_proba(X_test_prd)[:,1]
pred_lst=pred.tolist()

# Exporting sample output

sample_submission.loc[:,'Is_Lead']=pred_lst

#Final Submission file
sample_submission.to_csv('submission.csv',index=False)